# Preprocess calls

### Imports

In [1]:
import sys 
sys.path.append('/Users/Emily/code/emilycardwell/final-project-cleaning/py_files')

In [2]:
import pandas as pd
import os

from string_cleaning import clean_chords
from genre_cleaning import genre_cleaning
from columns import new_columns, song_length, drop_dups_cols, expand_cols
from get_data_slice import filter_length, get_songs
from utils import df_to_csv, count_chords, count_genres, count_artists, \
                  get_csv_data, get_text_data

## Read Data

In [ ]:
raw_kaggle_df = get_csv_data('kaggle_raw.csv')
len(raw_kaggle_df)

In [ ]:
raw_kaggle_df.head(3)

In [ ]:
raw_jazz_col = get_text_data('jazz_raw.txt')
raw_jazz_df = expand_cols(raw_jazz_col)
raw_jazz_df

## Drop Duplicates and unwanted columns (only if song/artist data exists)

In [ ]:
slim_df = drop_dups_cols(raw_kaggle_df)
len(slim_df)

## Concat DFs

In [ ]:
concat_df = pd.concat([slim_df, raw_jazz_df], ignore_index=True)
len(concat_df)

## Clean Chords

In [ ]:
cleaned_df = concat_df.copy()
cleaned_chords_column = clean_chords(concat_df['chords'])
cleaned_df['chords'] = cleaned_chords_column

### drop duplicates based on chords and song name

In [ ]:
new_columns_df = new_columns(cleaned_df)

unreplicated_df = \
    new_columns_df.drop_duplicates(subset=['chords_list', 'song_name'],
                                   keep = 'last').reset_index(drop = True)
    
# drop unnecessary columns
unreplicated_df.drop(columns=['song_name', 'chords_list'], inplace=True)

## Clean Genres

In [ ]:
slim_genres = genre_cleaning(unreplicated_df['genres'])
clean_genres_df = unreplicated_df.copy()
clean_genres_df['genres'] = slim_genres

## Get Song Length (chord count)

In [ ]:
song_len_df = song_length(clean_genres_df)
song_len_df

## Filter DF

#### by song length

In [ ]:
final_df = filter_length(song_len_df)
len(final_df)

#### by number of samples (songs)

In [ ]:
#final_df = get_songs(final_df, size=5000)

## Send df to cvs

In [ ]:
# df_to_csv(final_df, '3', '~/code/emilycardwell/final-project-data/data/clean')

# Utility Calls

In [ ]:
chord_count_df = count_chords(final_df, low_freq_to_remove=10, 
                              histplot=True, ascending=True)
chord_count_df

In [ ]:
genre_count_df = count_genres(final_df, histplot=True)

In [ ]:
artists_df = count_artists(final_df, histplot=True)